In [204]:
using DelimitedFiles
using PyPlot

┌ Info: Precompiling PyPlot [d330b81b-6aea-500a-939a-2ce795aea3ee]
└ @ Base loading.jl:1273
┌ Info: Installing matplotlib via the Conda matplotlib package...
└ @ PyCall /Users/gribeill/.julia/packages/PyCall/ttONZ/src/PyCall.jl:705
┌ Info: Running `conda install -y matplotlib` in root environment
└ @ Conda /Users/gribeill/.julia/packages/Conda/kLXeC/src/Conda.jl:112


Solving environment: ...working... done

## Package Plan ##

  environment location: /Users/gribeill/.julia/conda/3

  added / updated specs:
    - matplotlib


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cycler-0.10.0              |           py37_0          14 KB
    freetype-2.9.1             |       hb4e5f40_0         555 KB
    kiwisolver-1.1.0           |   py37h0a44026_0          54 KB
    libpng-1.6.37              |       ha441bb4_0         262 KB
    matplotlib-3.1.1           |   py37h54f8f79_0         4.9 MB
    pyparsing-2.4.5            |             py_0          62 KB
    python-dateutil-2.8.1      |             py_0         224 KB
    pytz-2019.3                |             py_0         231 KB
    tornado-6.0.3              |   py37h1de35cc_0         590 KB
    ------------------------------------------------------------
                                           To

┌ Warning: PyPlot is using tkagg backend, which is known to cause crashes on MacOS (#410); use the MPLBACKEND environment variable to request a different backend.
└ @ PyPlot /Users/gribeill/.julia/packages/PyPlot/4wzW1/src/init.jl:192


## Day 1

In [20]:
input = readdlm("inputs/day1.txt", ',', Int)
mapreduce(x -> floor(Int, x / 3) - 2, +, input)

3212842

In [26]:
function total_fuel(mass)
    tf = 0
    f(x) = floor(Int, x / 3) - 2
    m = mass
    while f(m) > 0
        ff = f(m)
        tf += ff
        m = ff
    end
    return tf
end
mapreduce(total_fuel, +, input)

4816402

## Day 2

In [65]:
function computer(program)
    pc = 1
    while program[pc] != 99
        opcode, x, y, r = program[pc:pc+3]
        if opcode == 1
            program[r+1] = program[x+1]+program[y+1]
        elseif opcode == 2
            program[r+1] = program[x+1]*program[y+1]
        else
            @error "Something went wrong! Got code $(opcode)."
        end
        pc += 4
    end
    return program
end

computer (generic function with 1 method)

In [68]:
input = readdlm("inputs/day2.txt", ',', Int)
input[2] = 12
input[3] = 2
result = computer(input)
print("Position 0 has: $(result[1])")

Position 0 has: 12490719

In [79]:
initial_state = readdlm("inputs/day2.txt", ',', Int)

function test_comp(input, a, b)
    input[2] = a
    input[3] = b
    return computer(input)[1]
end

test_comp (generic function with 1 method)

In [82]:
for a=0:99
    for b=0:99
        if 19690720 == test_comp(copy(initial_state), a, b)
            println("Found state: $(100*a + b).")
            break
        end
    end
end

Found state: 2003.


## Day 3

In [239]:
struct Point
    x::Int
    y::Int
end

function manhattan(a::Point, b::Point)
    return abs(a.x-b.x) + abs(a.y-b.y)
end

struct Segment
    start::Point
    stop::Point
end

function sorted_x(s::Segment)
    return sort([s.start.x, s.stop.x])
end

function sorted_y(s::Segment)
    return sort([s.start.y, s.stop.y])
end

function move(p::Point, dir::Symbol, d::Int)
    if dir == :R
        stop = Point(p.x+d, p.y)
    elseif dir == :L
        stop = Point(p.x-d, p.y)
    elseif dir == :U
        stop = Point(p.x, p.y+d)
    elseif dir == :D
        stop = Point(p.x, p.y-d)
    end
    return Segment(p, stop)
end

function wire_dir(s::Segment)
    if s.start.x == s.stop.x 
        return :V
    else
        return :H
    end
end

function intersect(s1::Segment, s2::Segment)
    no_intersection = (false, Point(0, 0))
    if wire_dir(s1) == wire_dir(s2)
        return no_intersection #for now assume collinear wires cannot intersect
    else
        if wire_dir(s1) == :H
            x1, x2 = sorted_x(s1)
            y1, y2 = sorted_y(s2)
            if (x1 <= s2.start.x <= x2) && (y1 <= s1.start.y <= y2)
                return (true, Point(s2.start.x, s1.start.y))
            else
                return no_intersection
            end
        else
            x1, x2 = sorted_x(s2)
            y1, y2 = sorted_y(s1)
            if (x1 <= s1.start.x <= x2) && (y1 <= s2.start.y <= y2)
                return (true, Point(s1.start.x, s2.start.y))
            else
                return no_intersection
            end
        end
    end
end

function parse_input_moves(input)
    moves = split(input, ',')
    return [(Symbol(m[1]), parse(Int, m[2:end])) for m in moves]
end

function build_wires(moves)
    wires = Array{Segment, 1}()
    start = Point(0,0)
    for m = moves
        new_wire = move(start, m...)
        push!(wires, new_wire)
        start = new_wire.stop
    end
    return wires
end     

function plot_wires(w1, w2)
    x = [0]
    y = [0]
    for w=w1
        push!(x, w.stop.x)
        push!(y, w.stop.y)
    end
    plot(x,y,"ro-")
    x = [0]
    y = [0]
    for w=w2
        push!(x, w.stop.x)
        push!(y, w.stop.y)
    end
    plot(x,y,"bo-")
end

function find_interection_distances(wires1, wires2)
    intersections = []
    for w1 = wires1
        for w2 = wires2
            test = intersect(w1, w2)
            if test[1] 
                push!(intersections, test[2])
                #print(test[2])
            end
        end
    end
    dst(p) = manhattan(p, Point(0,0))
    return sort([dst(p) for p in intersections if dst(p)>0])
end
                
function distance_along_wire(w::Segment, p::Point)
    return manhattan(w.start, p)
end
                
function wire_length(w::Segment)
    return manhattan(w.start, w.stop)
end
                                
function find_intersection_delays(wires1, wires2)
     delays = []
     for (j, w1) = enumerate(wires1)
        for (k, w2) = enumerate(wires2)
            test = intersect(w1, w2)
            if test[1]
                if test[2] == Point(0,0)
                    continue
                end
                #println(test[2])
                prev_delay_1 = mapreduce(wire_length, +, wires1[1:j-1])
                prev_delay_2 = mapreduce(wire_length, +, wires2[1:k-1])
                #println(prev_delay_1, " ", prev_delay_2)
                d1 = distance_along_wire(w1, test[2])
                d2 = distance_along_wire(w2, test[2])
                #println(d1, " ", d2)
                push!(delays, d1+d2+prev_delay_1+prev_delay_2)
            end
        end    
    end
    return sort(delays)                
end

find_intersection_delays (generic function with 1 method)

In [240]:
L1 = parse_input_moves("R8,U5,L5,D3")
L2 = parse_input_moves("U7,R6,D4,L4")
W1 = build_wires(L1)
W2 = build_wires(L2);
find_intersection_delays(W1,W2)

2-element Array{Any,1}:
 30
 40

In [241]:
L1 = parse_input_moves("R75,D30,R83,U83,L12,D49,R71,U7,L72")
L2 = parse_input_moves("U62,R66,U55,R34,D71,R55,D58,R83")
W1 = build_wires(L1)
W2 = build_wires(L2);
find_interection_distances(W1,W2);
find_intersection_delays(W1,W2)

4-element Array{Any,1}:
 610
 624
 726
 850

In [242]:
L1 = parse_input_moves("R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51")
L2 = parse_input_moves("U98,R91,D20,R16,D67,R40,U7,R15,U6,R7")
W1 = build_wires(L1)
W2 = build_wires(L2);
find_interection_distances(W1,W2);
find_intersection_delays(W1,W2)

5-element Array{Any,1}:
 410
 516
 636
 650
 700

In [245]:
input = readlines("inputs/day3.txt")
L1 = parse_input_moves(input[1])
L2 = parse_input_moves(input[2])
W1 = build_wires(L1)
W2 = build_wires(L2);
its = find_interection_distances(W1,W2);
println("Distance to nearest intersection: $(its[1])")
dels = find_intersection_delays(W1,W2);
println("Delay to nearest intersection: $(dels[1])")

Distance to nearest intersection: 1626
Delay to nearest intersection: 27330
